<h1 align="center"> Deep RL - A Deep Reinforcement Learning Framework </h1>

![](imgs/logo.png)

# Set the path of DeepRL

Before using any component of the library we need to add the framework to the path of Python interpreter. We append the location of the root folder of the framework to system path. In my case it is at the following path. After setting the path we import the RLExp class which helps us to create a Deep RL Experiment.

In [1]:
FRAMEWORK_PATH = "/home/mayank/Documents/Codes/ValueBased_DeepRL/"

In [2]:
import sys
import os
sys.path.append(FRAMEWORK_PATH)
from RLExp import RLExp

# Querying available implementations

The framework is under heavy development and we are adding new algorithms after testing everyday. The list of all available implementations of algorithm, exploration strategies, experience replays can be queried as demonstrated below.

In [3]:
exp = RLExp()

## 1. Get list of all available algorithms

In [4]:
exp.getAvailableAlgorithms()

['DQN', 'DDQN']

## 2. Get list of all implemented experience replays

In [5]:
exp.getAvailableReplays()

['Uniform-Sampling', 'Prioritized-Sampling']

## 3. Get list of all implemented exploration strategies

In [6]:
exp.getAvailableExplorePolicies()

['epsilon-greedy', 'annealing-ep-greedy']

# Setting Up RL Experiment

Now, we set up a RL experiment. To do that we need to call a function setup_exp of class RLExp which takes the following arguments as the input. In this step we will define the algorithm, exploration policy, replay method and neural network architecture we like to use for our experiment.

Arguments of setup_exp function
-  algorithm - one of the algorithm returned using getAvailableAlgorithms method
-  explore_policy - one of the exploration policy returned using getAvailableExplorePolicies method
-  replay - one of the replay returned using getAvailableReplays method
-  observation_shape - shape of the observation which is feed to the input of neural network
-  num_actions - the number of actions agent can perform in the environment

Further, to demonstrate we will train our agent on a simple cartpole environment from OpenAI Gym. Since, OpenAI exposes the size of Action and Observation so it becomes really easy to determine the shape of observation and num of actions which are required as the arguments to the setup_exp method.



In [7]:
import gym
from gym import wrappers
env = gym.make("CartPole-v0")
#env = wrappers.Monitor(env, 'data/cartpole-experiment-1')
observation_size = env.observation_space.shape[0] # observation should be numpy array/vector
num_actions = env.action_space
print("Observation Shape : ",observation_size)
print("Number of actions : ",num_actions)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Observation Shape :  4
Number of actions :  Discrete(2)


## Defining Neural Network Architecture

When we call setup_exp method it looks for the defination of neural network architecture which is compatible with provided observation_shape and num_actions. The network should be defined in the file provided in "network" folder. Please do not change the file name. File **q_network.py** should contain the network defination. The network is defined using PyTorch. Example given below demonstrates how to define a PyTorch Network.

In [8]:
! cd ..; tree network;

network
├── __pycache__
│   └── q_network.cpython-36.pyc
└── q_network.py

1 directory, 2 files


In [9]:
! cd ..; cd network; cat q_network.py

import torch.nn as nn
import torch

# Define your network here
# input output format should be same as provided here.
class q_network(nn.Module):

    def __init__(self,input_size,out_size):
        super(q_network,self).__init__()

        self.input_size = input_size
        self.out_size = out_size

        self.layer1 = nn.Linear(out_features=24,in_features=self.input_size)
        self.layer2 = nn.Linear(out_features=48,in_features=24)
        self.layer3 = nn.Linear(out_features=self.out_size,in_features=48)

        self.relu = nn.ReLU()

    def forward(self,x,bsize):

        x = x.view(bsize,self.input_size)
        q_out = self.relu(self.layer1(x))
        q_out = self.relu(self.layer2(q_out))
        q_out = self.layer3(q_out)

        return q_out


## Calling setup_exp method to create an experiment

Finally, once all the step given above are completed. We can call setup_exp function to create the experiment.

In [10]:
exp.setup_exp(algorithm="DDQN",explore_policy="annealing-ep-greedy",replay="Uniform-Sampling",observation_shape=observation_size,num_actions=2,exp_name='cartpole',save_interval=100)

Device Set to :  cpu
q_network(
  (layer1): Linear(in_features=4, out_features=24, bias=True)
  (layer2): Linear(in_features=24, out_features=48, bias=True)
  (layer3): Linear(in_features=48, out_features=2, bias=True)
  (relu): ReLU()
)
Algorithm Setup Done. Please set hyper-parameters.


# Setting hyper-parameters

An RL experiment have many hyper-parameters which are required to tuned in order to get the best performance from the algorithms. The Deep RL exposes all the hyper-parameters which can be changed and queried at any time during the experiment. The following methods can be used to get the all hyper-parameters of each component, followed by how to change them.

## Querying and Setting Algorithm hyper-parameters

In [11]:
exp.algorithm_object.getHyperParams()

Hyper Paramters (DDQN)					Current Value
-------------------------------------------------------------------
1. batch_size							None
2. gamma							None
3. freeze_steps							None
4. max_steps							None
5. max_episodes							None
6. optimizer							None
7. learn rate							None
8. loss func							None


In [12]:
exp.algorithm_object.setHyperParams(batch_size=32,gamma=0.99,freeze_steps=10000,max_steps=10**6,max_episodes=10000,optimizer='adam',lr=0.000625,loss="huber")

## Querying and Setting Exploration Policy hyper-parameters

In [13]:
exp.explore_policy_object.getHyperParams()

Hyper Paramters (annealed_ep_greedy)			Current Value
-------------------------------------------------------------------
1. current_epsilon					None
2. intital_epsilon					None
3. final_epsilon					None
4. episodes_to_anneal					None


In [14]:
exp.explore_policy_object.setHyperParams(episodes_to_anneal=7000,initial_epsilon=1.0,final_epsilon=0.01)

## Querying and Setting Replay hyper-parameters

In [15]:
exp.replay_object.getHyperParams()

Hyper Paramters (uniform-sampling)			Current Value
-------------------------------------------------------------------
1. capacity						None


In [16]:
exp.replay_object.setHyperParams(capacity=1000000)

## Printing Final values of hyper-parameters

In [17]:
exp.algorithm_object.getHyperParams()

Hyper Paramters (DDQN)					Current Value
-------------------------------------------------------------------
1. batch_size							32
2. gamma							0.99
3. freeze_steps							10000
4. max_steps							1000000
5. max_episodes							10000
6. optimizer							adam
7. learn rate							0.000625
8. loss func							huber


In [18]:
exp.explore_policy_object.getHyperParams()

Hyper Paramters (annealed_ep_greedy)			Current Value
-------------------------------------------------------------------
1. current_epsilon					1.0
2. intital_epsilon					1.0
3. final_epsilon					0.01
4. episodes_to_anneal					7000


In [19]:
exp.replay_object.getHyperParams()

Hyper Paramters (uniform-sampling)			Current Value
-------------------------------------------------------------------
1. capacity						1000000


# Training Loop

Since, we give the control of looping over training epochs to the programmer. He/She need to pass the current step and episode number whenever calling the RLExp class so that the data can be logged.

## Fill the memory with random transitions

Filling the replay memory before running the training loop is a good idea and hence, we force you to use fill the replay to atleast greater than batch size. If replay memory is smaller than the batch size an error is raised.

In [20]:
for episode in range(0,1000):
    
    prev_state = env.reset()
    
    for step in range(0,exp.algorithm_object.params["max_steps"]):
        
        action = env.action_space.sample()
        
        next_state,reward,done,_ = env.step(action)
        
        sample = exp.pack_sample(prev_state,action,reward,next_state,done)
        
        exp.replay_object.add_sample(sample)
        
        prev_state = next_state
        
        if done:
            break

## Important Note

Please don't rerun this block of the code if it is once executed. Restart from start.

In [21]:
for epsiode in range(0,exp.algorithm_object.params["max_episodes"]):
    
    episode_reward = 0.0
    prev_state = env.reset()
    #env.render()
    
    for step in range(0,exp.algorithm_object.params["max_steps"]):
        
        # get an action
        action = exp.explore_policy_object.exploreAction(state=prev_state,curr_epsiode=epsiode)
        
        # perform action in environment
        next_state,reward,done,_ = env.step(action)
        
        # pack the data into a named tuple provided by the framework
        sample = exp.pack_sample(prev_state,action,reward,next_state,done)
        
        # add sequence to experience replay
        exp.replay_object.add_sample(sample)
        
        episode_reward += reward
        prev_state = next_state
        
        exp.algorithm_object.update()
        
        # important
        exp.time_steps_elapsed += 1
        
        # copy weights
        if (exp.time_steps_elapsed % exp.algorithm_object.params["freeze_steps"]) == 0:
            exp.algorithm_object.copytoTarget()
        
        if done:
            #print(done,step+1)
            break
    
    # store episode reward
    exp.algorithm_object.store_episode_reward(episode_reward)
    
    # update epsilon
    exp.explore_policy_object.updateEpsilon()
    
    if (epsiode+1) % exp.save_interval == 0:
        exp.dumpData_object.export_csv()
    
    print('Current Exploration Rate : %.4f'%(exp.explore_policy_object.params["current_epsilon"]))

Current Exploration Rate : 0.9999
Current Exploration Rate : 0.9997
Current Exploration Rate : 0.9996
Current Exploration Rate : 0.9994
Current Exploration Rate : 0.9993
Current Exploration Rate : 0.9992
Current Exploration Rate : 0.9990
Current Exploration Rate : 0.9989
Current Exploration Rate : 0.9987
Current Exploration Rate : 0.9986
Current Exploration Rate : 0.9984
Current Exploration Rate : 0.9983
Current Exploration Rate : 0.9982
Current Exploration Rate : 0.9980
Current Exploration Rate : 0.9979
Current Exploration Rate : 0.9977
Current Exploration Rate : 0.9976
Current Exploration Rate : 0.9975
Current Exploration Rate : 0.9973
Current Exploration Rate : 0.9972
Current Exploration Rate : 0.9970
Current Exploration Rate : 0.9969
Current Exploration Rate : 0.9967
Current Exploration Rate : 0.9966
Current Exploration Rate : 0.9965
Current Exploration Rate : 0.9963
Current Exploration Rate : 0.9962
Current Exploration Rate : 0.9960
Current Exploration Rate : 0.9959
Current Explor

Current Exploration Rate : 0.9651
Current Exploration Rate : 0.9649
Current Exploration Rate : 0.9648
Current Exploration Rate : 0.9646
Current Exploration Rate : 0.9645
Current Exploration Rate : 0.9644
Current Exploration Rate : 0.9642
Current Exploration Rate : 0.9641
Current Exploration Rate : 0.9639
Current Exploration Rate : 0.9638
Current Exploration Rate : 0.9637
Current Exploration Rate : 0.9635
Current Exploration Rate : 0.9634
Current Exploration Rate : 0.9632
Current Exploration Rate : 0.9631
Current Exploration Rate : 0.9629
Current Exploration Rate : 0.9628
Current Exploration Rate : 0.9627
Current Exploration Rate : 0.9625
Current Exploration Rate : 0.9624
Current Exploration Rate : 0.9622
Current Exploration Rate : 0.9621
Current Exploration Rate : 0.9620
Current Exploration Rate : 0.9618
Current Exploration Rate : 0.9617
Current Exploration Rate : 0.9615
Current Exploration Rate : 0.9614
Current Exploration Rate : 0.9612
Current Exploration Rate : 0.9611
Current Explor

Current Exploration Rate : 0.9306
Current Exploration Rate : 0.9304
Current Exploration Rate : 0.9303
Current Exploration Rate : 0.9301
Current Exploration Rate : 0.9300
Current Exploration Rate : 0.9299
Current Exploration Rate : 0.9297
Current Exploration Rate : 0.9296
Current Exploration Rate : 0.9294
Current Exploration Rate : 0.9293
Current Exploration Rate : 0.9291
Current Exploration Rate : 0.9290
Current Exploration Rate : 0.9289
Current Exploration Rate : 0.9287
Current Exploration Rate : 0.9286
Current Exploration Rate : 0.9284
Current Exploration Rate : 0.9283
Current Exploration Rate : 0.9282
Current Exploration Rate : 0.9280
Current Exploration Rate : 0.9279
Current Exploration Rate : 0.9277
Current Exploration Rate : 0.9276
Current Exploration Rate : 0.9274
Current Exploration Rate : 0.9273
Current Exploration Rate : 0.9272
Current Exploration Rate : 0.9270
Current Exploration Rate : 0.9269
Current Exploration Rate : 0.9267
Current Exploration Rate : 0.9266
Current Explor

Current Exploration Rate : 0.8963
Current Exploration Rate : 0.8962
Current Exploration Rate : 0.8960
Current Exploration Rate : 0.8959
Current Exploration Rate : 0.8958
Current Exploration Rate : 0.8956
Current Exploration Rate : 0.8955
Current Exploration Rate : 0.8953
Current Exploration Rate : 0.8952
Current Exploration Rate : 0.8951
Current Exploration Rate : 0.8949
Current Exploration Rate : 0.8948
Current Exploration Rate : 0.8946
Current Exploration Rate : 0.8945
Current Exploration Rate : 0.8944
Current Exploration Rate : 0.8942
Current Exploration Rate : 0.8941
Current Exploration Rate : 0.8939
Current Exploration Rate : 0.8938
Current Exploration Rate : 0.8936
Current Exploration Rate : 0.8935
Current Exploration Rate : 0.8934
Current Exploration Rate : 0.8932
Current Exploration Rate : 0.8931
Current Exploration Rate : 0.8929
Current Exploration Rate : 0.8928
Current Exploration Rate : 0.8927
Current Exploration Rate : 0.8925
Current Exploration Rate : 0.8924
Current Explor

Current Exploration Rate : 0.8621
Current Exploration Rate : 0.8620
Current Exploration Rate : 0.8618
Current Exploration Rate : 0.8617
Current Exploration Rate : 0.8615
Current Exploration Rate : 0.8614
Current Exploration Rate : 0.8613
Current Exploration Rate : 0.8611
Current Exploration Rate : 0.8610
Current Exploration Rate : 0.8608
Current Exploration Rate : 0.8607
Current Exploration Rate : 0.8606
Current Exploration Rate : 0.8604
Current Exploration Rate : 0.8603
Current Exploration Rate : 0.8601
Current Exploration Rate : 0.8600
Current Exploration Rate : 0.8598
Current Exploration Rate : 0.8597
Current Exploration Rate : 0.8596
Current Exploration Rate : 0.8594
Current Exploration Rate : 0.8593
Current Exploration Rate : 0.8591
Current Exploration Rate : 0.8590
Current Exploration Rate : 0.8589
Current Exploration Rate : 0.8587
Current Exploration Rate : 0.8586
Current Exploration Rate : 0.8584
Current Exploration Rate : 0.8583
Current Exploration Rate : 0.8581
Current Explor

...

Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Explor

Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Explor

Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Explor

Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Exploration Rate : 0.0100
Current Explor

KeyboardInterrupt: 

In [22]:
import torch
import numpy as np

# Performance of the Agent

In [23]:
# Calculate Performance of the agent
test_episodes = 100
reward_list = []

In [24]:
for episode in range(0,test_episodes):
    
    prev_state = env.reset()
    total_reward = 0
    
    for step in range(0,1000):
        
        with torch.no_grad():
            torch_x = torch.from_numpy(prev_state).to(exp.device).float()
            out = exp.main_model.forward(torch_x,bsize=1)
            val, action = out.max(dim=1)
            action = int(action)
        
        next_state,reward,done,_ = env.step(action)
        
        total_reward += reward
        
        prev_state = next_state
        
        if done:
            break
    reward_list.append(total_reward)
print('Test Done')

Test Done


In [26]:
reward_list = np.array(reward_list)
print('Mean Reward : ',reward_list.mean())
print('Std. Deviation : ',reward_list.std())
print('Max Reward : ',reward_list.max())
print('Min Reward : ',reward_list.min())

Mean Reward :  72.63
Std. Deviation :  13.852548501990526
Max Reward :  92.0
Min Reward :  42.0
